In [1]:
import pandas as pd
import numpy as np
import re
from time import time
import sys
import pickle
import os

from keras.engine.topology import Layer
from keras.layers import InputSpec
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Flatten, Lambda
from keras.layers import Conv2D, MaxPool2D, MaxPooling2D

from keras import backend as K
from keras.utils import conv_utils, to_categorical
from keras.datasets import mnist

import tensorflow as tf
from tensorflow.python.framework import ops  
from tensorflow.python.ops import gen_nn_ops

import math
import random
random.seed(91)

/home/syht/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
batch_size = 8
num_classes = 10
epochs = 1

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [3]:
def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(16, (3, 3), activation='relu'))
    model.add(StochasticPooling(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    model.summary()
    return model

In [197]:
class StochasticPooling(Layer):
    def __init__(self, pool_size=(2, 2), padding='SAME', data_format='channels_last', **kwargs):
        super(StochasticPooling, self).__init__(**kwargs)
        
        self.pool_size = conv_utils.normalize_tuple(pool_size, 2, 'pool_size')
        self.strides = conv_utils.normalize_tuple(pool_size, 2, 'strides')
        self.padding = padding
        self.data_format = 'NHWC' if data_format=='channels_last' else 'NCHW' 
        #conv_utils.normalize_data_format(data_format)
        self.input_spec = InputSpec(ndim=4)
        
    def _tf_pooling_function(self, x, name=None):
        
        def _pool(inputs, is_train):
            start = time()
            #assert np.min(inputs) > 0
            ### min input into 0
            inputs[inputs < 0] = 0
            if np.sum(inputs) == 0:
                inputs = np.random.rand(inputs.shape)
            
            input_shape = inputs.shape
            batch, r, c, channel = input_shape[0], input_shape[1], input_shape[2], input_shape[3]
            #print('Shape:', input_shape)

            pr, pc = self.pool_size 
            sr, sc = self.strides

            # compute number of windows
            num_r =  math.ceil(r/sr) if self.padding == 'SAME' else r//sr
            num_c = math.ceil(c/sc) if self.padding == 'SAME' else c//sc
            
            # reshape
            w = np.transpose(inputs, (0, 3, 1, 2)) 
            w = np.reshape(w, (batch*channel, r, c))

            def pool(x):                
                if np.sum(x) == 0:
                    cache = np.zeros((pr, pc))
                    cache[0, 0] = 1.0
                    return cache

                x_prob = x / np.sum(x)
                
                if is_train:
                    ### in forward pass
                    # sort
                    size = pr*pc
                    x_prob = np.reshape(x_prob, [size])
                    x_sorted = np.argsort(-x_prob)

                    while True:
                        h = random.randint(0, size - 1)
                        y = random.random()

                        p = x_sorted[h]

                        if x_prob[p] >= y:
                            break

                    pool_matrix = np.zeros((pr, pc))
                    pool_matrix[p//pr, p%pc] = 1.0

                    return pool_matrix
                else:
                    return x_prob

            re = np.zeros((batch*channel, num_r, num_c), dtype=np.float32)
            # extract with pool_size
            for i in range(num_r):
                for j in range(num_c):
                    crop = np.zeros((batch*channel, pr, pc))
                    crop[:,:,:] = w[:, i*sr:i*sr+pr, j*sc:j*sc+pc]

                    # pool
                    outs = np.array(list(map(pool, crop)))
                    
                    if is_train:
                        re[:, i, j] = (crop * outs).max(axis=(1,2))
                    else:
                        re[:, i, j] = (crop * outs).sum(axis=(1,2))

            # reshape
            re = np.reshape(re, (batch, channel, num_r, num_c))
            re = np.transpose(re, (0, 2, 3, 1))
            
            #print(re.shape)
            #print('Pool time:', time() - start)
            #print(is_train)
            return re

        def custom_grad(op, grad):
            if self.data_format=='NHWC':
                ksizes=[1, self.pool_size[0], self.pool_size[1], 1]
                strides=[1, self.strides[0], self.strides[1], 1]
            else:
                ksizes=[1, 1, self.pool_size[0], self.pool_size[1]]
                strides=[1, 1, self.strides[0], self.strides[1]]
            return gen_nn_ops.max_pool_grad_v2(
                op.inputs[0],
                op.outputs[0],
                grad,
                ksizes,
                strides,
                self.padding,
                data_format=self.data_format
            ), K.tf.constant(0.0)
            
            """
            inputs = op.inputs[0]
            outputs = op.outputs[0]
            
            batch = K.shape(inputs)[0]
            input_shape = K.int_shape(inputs)
            r, c, channel = input_shape[1], input_shape[2], input_shape[3]
            
            pr, pc = self.pool_size 
            sr, sc = self.strides

            # compute number of windows
            num_r =  math.ceil(r/sr) if self.padding == 'SAME' else r//sr
            num_c = math.ceil(c/sc) if self.padding == 'SAME' else c//sc
            
            # grad return
            grad_re = K.tf.zeros_like(inputs)
            
            # transpose
            grad_re = K.transpose(grad_re, perm=[0, 3, 1, 2])
            inputs = K.transpose(inputs, perm=[0, 3, 1, 2])
            outputs = K.transpose(outputs, perm=[0, 3, 1, 2])
            
            for i in range(num_r):
                for j in range(num_c):
                    crop = np.zeros((batch*channel, pr, pc))
                    crop[:,:,:] = w[:, i*sr:i*sr+pr, j*sc:j*sc+pc]

                    # pool
                    outs = np.array(list(map(pool, crop)))
                    
                    if is_train:
                        re[:, i, j] = (crop * outs).max(axis=(1,2))
                    else:
                        re[:, i, j] = (crop * outs).sum(axis=(1,2))
            
            
            
            
            
            mask = K.tf.where(K.tf.equal())
            
            
            return grad_re, K.tf.constant(0.0)
            """

        def py_func(func, inp, Tout, stateful=True, name=None, grad=None):
            # Need to generate a unique name to avoid duplicates:
            rnd_name = 'StochasticPoolingGrad' + str(np.random.randint(0, 1E+8))

            K.tf.RegisterGradient(rnd_name)(grad)  
            g = K.tf.get_default_graph()
            with g.gradient_override_map({"PyFunc": rnd_name}):
                return K.tf.py_func(func, inp, Tout, stateful=stateful, name=name)
        
        with ops.name_scope(name, "mod", [x]) as name:
            z = py_func(_pool,
                        [x, K.tf.stop_gradient(K.in_train_phase(1, 0))],
                        [K.tf.float32],
                        name=name,
                        grad=custom_grad)
            
            return z[0]

    def compute_output_shape(self, input_shape):
        r, c = input_shape[1], input_shape[2]
        sr, sc = self.strides
        num_r =  math.ceil(r/sr) if self.padding == 'SAME' else r//sr
        num_c = math.ceil(c/sc) if self.padding == 'SAME' else c//sc
        return (input_shape[0], num_r, num_c, input_shape[3])
    
    def call(self, inputs):
        output = self._tf_pooling_function(inputs) #K.in_train_phase(self._tf_pooling_function(inputs), self._pooling_function_test(inputs))
        return output
    
    def get_config(self):
        config = {
                  'pool_size': self.pool_size,
                  'strides': self.strides
                }
        base_config = super(StochasticPooling, self).get_config()
        return dict(list(base_config.items()) + list(config.items())) 

In [198]:
model = get_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_97 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_98 (Conv2D)           (None, 24, 24, 16)        4624      
_________________________________________________________________
stochastic_pooling_49 (Stoch (None, 12, 12, 16)        0         
_________________________________________________________________
dropout_91 (Dropout)         (None, 12, 12, 16)        0         
_________________________________________________________________
flatten_46 (Flatten)         (None, 2304)              0         
_________________________________________________________________
dense_91 (Dense)             (None, 128)               295040    
_________________________________________________________________
dropout_92 (Dropout)         (None, 128)               0         
__________

In [ ]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
 1640/60000 [..............................] - ETA: 1:14:35 - loss: 1.1199 - acc: 0.6250

In [ ]:
def TestStochasticPoolModel():
    inputs = Input(shape=(65, 65, 3), name='input', dtype='float32')
    conv = Conv2D(32, (3, 3), activation='relu')(inputs)
    pool = StochasticPooling(padding='VALID')(conv)
    #pool = MaxPool2D()(conv)
    flat = Flatten()(pool)
    dense = Dense(32, activation='relu')(flat)
    out = Dense(1, activation='sigmoid')(dense)
    
    model = Model(inputs, out)
    
    model.summary()
    
TestStochasticPoolModel()

In [30]:
inputs = np.random.rand(128, 24, 24, 16)

padding = 'SAME'
batch, r, c, channel = inputs.shape[0], inputs.shape[1], inputs.shape[2], inputs.shape[3]
pr = 2
pc = 2
sr, sc = 2, 2
num_r =  math.ceil(r/sr) if padding == 'SAME' else r//sr
num_c = math.ceil(c/sc) if padding == 'SAME' else c//sc

start = time()

# reshape
w = np.transpose(inputs, (0, 3, 1, 2)) 
w = np.reshape(w, (batch*channel, r, c))
print(w.shape)

def pool(x):
    x_prob = x / np.sum(x)

    ### in forward pass
    # sort
    size = pr*pc
    x_prob = np.reshape(x_prob, [size])
    x_sorted = np.argsort(-x_prob)

    #print(x_prob)
    while True:
        h = random.randint(0, size - 1)
        y = random.random()

        p = x_sorted[h]

        #print(h, y, p)
        if x_prob[p] >= y:
            break

    cache = np.zeros((pr, pc))
    cache[p//pr, p%pc] = 1.0

    return cache #(np.reshape(x, [size])[p], 

re = np.zeros((batch*channel, num_r, num_c), dtype=np.float32)
re_p = np.zeros((batch*channel, num_r, num_c, pr, pc), dtype=np.float32)
# extract with pool_size
for i in range(num_r):
    for j in range(num_c):
        crop = np.zeros((batch*channel, pr, pc))
        crop[:,:,:] = w[:, i*sr:i*sr+pr, j*sc:j*sc+pc]

        # pool
        outs = np.array(list(map(pool, crop)))

        re[:, i, j] = (crop * outs).max(axis=(1,2))
        re_p[:, i, j] = outs

# reshape
re = np.reshape(re, (batch, channel, num_r, num_c))
re = np.transpose(re, (0, 2, 3, 1))

print(re_p.shape)


print(time() - start)

(2048, 24, 24)
(2048, 12, 12, 2, 2)
13.69327688217163


In [ ]:
        def pooling_function(inputs):
            """
            Parameters
            ----------
            inputs : 4-tensor in format (batch size, rows, cols, channels), 
                    IMPORTANT: All values should be positive
            """
            inputs[inputs < 0] = 0
            if np.sum(inputs) == 0:
                inputs = np.random.rand(inputs.shape)
            
            # get image shape
            input_shape = inputs.shape
            batch, r, c, channel = input_shape[0], input_shape[1], input_shape[2], input_shape[3]

            pr, pc = self.pool_size 
            sr, sc = self.strides

            # compute number of windows
            num_r =  math.ceil(r/sr) if self.padding == 'SAME' else r//sr
            num_c = math.ceil(c/sc) if self.padding == 'SAME' else c//sc

            # apply pool
            def stochastic_pool(x_crop):
                #### calculate probabilities
                x_prob = x_crop / K.sum(x_crop)

                ### in forward pass
                # sort
                size = pr*pc
                x_prob = K.reshape(x_prob, [size])
                
                x_sorted = K.tf.nn.top_k(x_prob, k=size, sorted=True).indices

                # sample a position
                y = 1.0 #K.tf.constant(1.0)
                z_train = 0.0 # K.tf.constant(0.0)
                x_position = 0 #K.tf.constant(0)

                def sample(y, z_train, _):
                    h = K.tf.constant(random.randint(0, size - 1))
                    y = K.tf.constant(random.random())
                    
                    x_position = K.cast(K.tf.where(K.tf.equal(x_sorted, h))[0][0], 'int32')
                        
                    z_train = x_prob[x_position]
                    
                    return y, z_train, x_position
                
                _, _, x_position = K.tf.while_loop(lambda y, z_train, _: K.tf.less(z_train, y), sample, [y, z_train, x_position])

                z = K.reshape(x_crop, [size])[x_position]
                return z
            
            # transpose inputs into [batch, channel, col, row]
            windows = K.tf.extract_image_patches(inputs, 
                                                ksizes=[1, pr, pc, 1], 
                                                strides=[1, sr, sc, 1], 
                                                rates=[1, 1, 1, 1], 
                                                padding=self.padding)

            # reshape       
            windows = K.reshape(windows, [-1, pr, pc, channel])
            windows = K.tf.transpose(windows, perm=[0, 3, 1, 2]) 
            windows = K.reshape(windows, [batch*num_r*num_c*channel, pr, pc])

            windows = K.tf.map_fn(stochastic_pool, windows)  
            
            # reshape
            windows = K.reshape(windows, [batch*num_r*num_c, channel, 1])
            windows = K.tf.transpose(windows, perm=[0, 2, 1])
            windows = K.reshape(windows, [batch, num_r, num_c, channel])
            
            result = K.tf.Session(config=K.tf.ConfigProto(log_device_placement=True)).run(windows)
            
            return result

In [ ]:
def _pooling_function_test(self, inputs):
        """
        Parameters
        ----------
        inputs : 4-tensor
            in format (batch size, rows, cols, channels),
            IMPORTANT: All values should be positive
        """
        # get image shape
        print(type(inputs))
        batch = K.shape(inputs)[0]
        input_shape = K.int_shape(inputs)
        r, c, channel = input_shape[1], input_shape[2], input_shape[3]
        
        pr, pc = self.pool_size 
        sr, sc = self.strides
        
        # compute number of windows
        num_r =  math.ceil(r/sr) if self.padding == 'SAME' else r//sr
        num_c = math.ceil(c/sc) if self.padding == 'SAME' else c//sc
        
        def _pool(x_crop):
            #### calculate probabilities
            x_prob = x_crop / K.sum(x_crop)
            
            ### in test phase, only take element wise multiply with the probabilities
            return K.sum(K.tf.multiply(x_crop, x_prob))
        
        # transpose inputs into [batch, channel, col, row]
        windows = K.tf.extract_image_patches(inputs, ksizes=[1, pr, pc, 1], 
                                          strides=[1, sr, sc, 1], rates=[1, 1, 1, 1], padding=self.padding)
        
        # reshape
        origin_shape = K.shape(windows)        
        windows = K.reshape(windows, [-1, pr, pc, channel])
        windows = K.tf.transpose(windows, perm=[0, 3, 1, 2]) 
        windows = K.reshape(windows, [batch*num_r*num_c*channel, pr, pc])
                
        # apply pool
        windows = K.tf.map_fn(_pool, windows)            
        
        # reshape
        windows = K.reshape(windows, [batch*num_r*num_c, channel, 1])
        windows = K.tf.transpose(windows, perm=[0, 2, 1])
        windows = K.reshape(windows, [batch, num_r, num_c, channel])
        
        return windows

In [175]:
a = K.tf.constant(np.random.rand(4,4))
print(K.eval(a))
b = K.tf.constant([[a[1,1], a[1,3]], [a[3,1], a[3,3]]])
print(K.eval(b))

#a = K.reshape(a, [4])
zeros = tf.zeros_like(a)
ones = tf.ones_like(a)

loc = K.tf.where(K.tf.equal(a, b), ones, zeros)
print(K.eval(loc))

K.eval(tf.boolean_mask(a,loc))

[[0.27348862 0.1686713  0.48607208 0.88841854]
 [0.15815645 0.66154016 0.43014945 0.89101383]
 [0.91413037 0.09411472 0.38693612 0.45047001]
 [0.2543991  0.25829275 0.51979136 0.09028198]]


TypeError: List of Tensors when single Tensor expected

In [184]:
a = tf.constant([[[1, 3], [2, 5]], [[3, 1], [4, 7]]])
b = tf.constant([1, 4])
K.eval(tf.where(tf.equal(a, b), a, tf.zeros_like(a)))

array([[[1, 0],
        [0, 0]],

       [[0, 0],
        [0, 0]]], dtype=int32)